+ def **UserForGenre( *`genero` : str* )**:
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf,
			     "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [1]:
#Importar librerias
import pandas as pd
import numpy as np 
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
#Traigo la tabla user_items para unirlas
df_user_items = pd.read_parquet('C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/ArchivosFinales/dfitems.parquet', engine='pyarrow')
df_user_items.head(2)


,item_id,item_name,playtime_forever,user_id,steam_id,items_count
0,10,Counter-Strike,6,76561197970982479,76561197970982479,277
1,20,Team Fortress Classic,0,76561197970982479,76561197970982479,277


In [3]:
df_user_items['user_id']=df_user_items['user_id'].astype('string') #paso de object a string

In [4]:
play_time_user= df_user_items[["item_id","playtime_forever","user_id"]] # me quedo con las variables necesarias
play_time_user.head(3)

,item_id,playtime_forever,user_id
0,10,6,76561197970982479
1,20,0,76561197970982479
2,30,7,76561197970982479


In [5]:
#Traigo la tabla steam games la cual voy a cruzar con la de arriba.con el item_id
df_games = pd.read_parquet('C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/ArchivosFinales/dfgamesrecomendacion.parquet', engine='pyarrow')
df_games.head(2)

,item_id,app_name,title,genres,release_date,developer,price
0,761140,Lost Summoner Kitty,Lost Summoner Kitty,Simulation,2018,Kotoshiro,4.99
1,761140,Lost Summoner Kitty,Lost Summoner Kitty,Casual,2018,Kotoshiro,4.99


In [7]:
# pongo la columnas que voy a usar.
tabladgenre = df_games[["item_id","genres","release_date"]]

In [8]:
#unifico
genre_user= play_time_user.merge(tabladgenre, on="item_id")
genre_user.head(5)

,item_id,playtime_forever,user_id,genres,release_date
0,10,6,76561197970982479,e-sports,2000
1,10,6,76561197970982479,1980s,2000
2,10,6,76561197970982479,Action,2000
3,10,6,76561197970982479,Score Attack,2000
4,10,6,76561197970982479,Team-Based,2000


In [9]:
genre_user['genres']=genre_user['genres'].astype('string') #pasar de object a string
genre_user['release_date']=genre_user['release_date'].astype('string')
genre_user['user_id']=genre_user['user_id'].astype('string')

In [10]:
#agrupo el genero por usuario y horas jugadas
user_hours_year= genre_user.groupby(["genres","user_id","release_date"])["playtime_forever"].sum().reset_index()
user_hours_year

,genres,user_id,release_date,playtime_forever
0,,10Degree,2016,0
1,,291129112911291129112911,2008,1795
2,,76561197981203305,2007,86
3,,76561197981203305,2009,0
4,,76561197990895276,2008,0
...,...,...,...,...
32987916,e-sports,zzyfo,2000,5
32987917,e-sports,zzyfo,2004,203
32987918,e-sports,zzyfo,2010,23
32987919,e-sports,zzyfo,2012,9308


In [11]:
#necesito saber los usuarios con mayor horas jugadas por genero
user_hours_total= genre_user.groupby(["genres","user_id"])["playtime_forever"].sum().reset_index()
user_hours_total.head(3)

,genres,user_id,playtime_forever
0,,10Degree,0
1,,291129112911291129112911,1795
2,,76561197981203305,86


In [12]:
#por cada genero el total de horas sumadas
max_user = user_hours_total.groupby(["genres"])["playtime_forever"].agg(playtime_forever="max").reset_index()
max_user

,genres,playtime_forever
0,,2353
1,1980s,358429
2,1990's,378184
3,2.5D,119709
4,2D,758893
...,...,...
333,World War I,117458
334,World War II,307674
335,Wrestling,18181
336,Zombies,488384


In [13]:
tabla_user =pd.merge(max_user,user_hours_total)
tabla_user #ya tengo el usuario con mayor horas jugadas por genero, ahora me falta incluir el total por año

,genres,playtime_forever,user_id
0,,2353,helenangel
1,1980s,358429,jiumohoo
2,1990's,378184,DownSyndromeKid
3,2.5D,119709,poadster
4,2D,758893,REBAS_AS_F-T
...,...,...,...
333,World War I,117458,DrQuaz
334,World War II,307674,Terminally-Chill
335,Wrestling,18181,boydeer
336,Zombies,488384,stopgovtcorruption


In [14]:
user_hours_year2=user_hours_year.rename(columns={'playtime_forever':'playtime_forever_year'})
user_hours_year2.head(5)

,genres,user_id,release_date,playtime_forever_year
0,,10Degree,2016,0
1,,291129112911291129112911,2008,1795
2,,76561197981203305,2007,86
3,,76561197981203305,2009,0
4,,76561197990895276,2008,0


In [15]:
#voy a probar si el cruce funciona ok, en el caso del usuario REBAS_AS_F-T	
prueba3=user_hours_year2[(user_hours_year2['user_id'] == 'REBAS_AS_F-T') & (user_hours_year2['genres']=='Indie')]
prueba3
#Para el usuario REBAS_AS_F-T la tabla tabla_user me debe traer 16 registros

,genres,user_id,release_date,playtime_forever_year
14873366,Indie,REBAS_AS_F-T,1999,0
14873367,Indie,REBAS_AS_F-T,2001,11
14873368,Indie,REBAS_AS_F-T,2003,1863
14873369,Indie,REBAS_AS_F-T,2005,2176
14873370,Indie,REBAS_AS_F-T,2006,1673
14873371,Indie,REBAS_AS_F-T,2007,1070
14873372,Indie,REBAS_AS_F-T,2008,1366
14873373,Indie,REBAS_AS_F-T,2009,28993
14873374,Indie,REBAS_AS_F-T,2010,21687
14873375,Indie,REBAS_AS_F-T,2011,106083


In [16]:
tabla_user2 =pd.merge(tabla_user,user_hours_year2)
tabla_user2

,genres,playtime_forever,user_id,release_date,playtime_forever_year
0,,2353,helenangel,2009,2353
1,1980s,358429,jiumohoo,1988,128
2,1980s,358429,jiumohoo,2000,356217
3,1980s,358429,jiumohoo,2014,743
4,1980s,358429,jiumohoo,2015,1098
...,...,...,...,...,...
1847,Zombies,488384,stopgovtcorruption,2013,165
1848,Zombies,488384,stopgovtcorruption,2014,8561
1849,Zombies,488384,stopgovtcorruption,2015,2436
1850,e-sports,721114,crustyoldkiller,2004,388621


In [17]:
#verificamos
prueba4=tabla_user2[(tabla_user2['user_id'] == 'REBAS_AS_F-T') & (tabla_user2['genres']=='Indie')]
prueba4.shape # esta ok
#ya tenemos la tabla para crear la funcion

(16, 5)

In [18]:
tabla_user2 = tabla_user2.rename(columns={'ref':'release_date', 'playtime_forever_year':'Horas jugadas'})
tabla_user2["release_date"] = pd.to_numeric(tabla_user2["release_date"])

In [19]:
def UserForGenre(genero):
    usuario= tabla_user2[tabla_user2["genres"]== genero]["user_id"].iloc[0] #obtengo usuario
    historial=tabla_user2[(tabla_user2['user_id'] == usuario) & (tabla_user2['genres']==genero)] #filtro por el genero y usuario
    historial2 = historial[['release_date', 'Horas jugadas']].copy() #me quedo con las columnas necesarias
    historial3=historial2.to_dict(orient="records")
    return {"Usuario":usuario ,"con más horas jugadas para": genero, "Historial acumulado": historial3 }

In [20]:
UserForGenre('Racing') # si funciona la funcion 

{'Usuario': 'DownSyndromeKid',
 'con más horas jugadas para': 'Racing',
 'Historial acumulado': [{'release_date': 1994, 'Horas jugadas': 0},
  {'release_date': 1995, 'Horas jugadas': 318},
  {'release_date': 1996, 'Horas jugadas': 0},
  {'release_date': 1999, 'Horas jugadas': 0},
  {'release_date': 2005, 'Horas jugadas': 91},
  {'release_date': 2006, 'Horas jugadas': 74553},
  {'release_date': 2007, 'Horas jugadas': 147814},
  {'release_date': 2008, 'Horas jugadas': 147707},
  {'release_date': 2009, 'Horas jugadas': 0},
  {'release_date': 2010, 'Horas jugadas': 0},
  {'release_date': 2011, 'Horas jugadas': 295361},
  {'release_date': 2013, 'Horas jugadas': 74271},
  {'release_date': 2014, 'Horas jugadas': 990},
  {'release_date': 2015, 'Horas jugadas': 4539},
  {'release_date': 2016, 'Horas jugadas': 185}]}

In [21]:
#para guardarla en la API 
tabla_user3 = "funcion_3.csv"
tabla_user2.to_csv(tabla_user3, index=False, encoding="utf-8")

In [23]:
#guardo parquet
funcion_3= pd.read_csv("funcion_3.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/Funciones/funcion_3.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(funcion_3)
pq.write_table(table,output_file)